In [2]:
# coding: utf-8
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize

import os
import random

In [3]:
negative = [f for f in os.listdir('./review_polarity 2/txt_sentoken/neg') if f.endswith('.txt')]

In [4]:
positive = [f for f in os.listdir('./review_polarity 2/txt_sentoken/pos') if f.endswith('.txt')]

In [5]:
arr_reviews = []

for review in negative:
    with open('./review_polarity 2/txt_sentoken/neg/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '0'
        dicti['text'] = text
        arr_reviews.append(dicti)
        
for review in positive:
    with open('./review_polarity 2/txt_sentoken/pos/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '1'
        dicti['text'] = text
        arr_reviews.append(dicti)

        
random.shuffle(arr_reviews)  # перемешиваем данные
    
df = pd.DataFrame(arr_reviews)  # создаем датасет


In [6]:
df.head()

,class,text
0,0,"and i thought "" stigmata "" would be the worst ..."
1,1,'lake placid' is definately not your typical c...
2,1,"jacques tati's 1953 classic "" les vacances de ..."
3,1,"seen september 13 , 1998 at 4 p . m at rotterd..."
4,0,my first exposure to the nightmare on elm stre...


In [7]:
df.describe()

,class,text
count,2000,2000
unique,2,2000
top,1,"this is your definitive "" hollywood "" movie , ..."
freq,1000,1


In [8]:
df.groupby('class').describe()

text
class                                                          
0     count                                                1000
      unique                                               1000
      top     this is your definitive " hollywood " movie , ...
      freq                                                    1
1     count                                                1000
      unique                                               1000
      top     synopsis : retiring detective jerry black ( ni...
      freq                                                    1

In [9]:
#  Делим выборку в соотновении 80:20
rev_train, rev_test, class_train, class_test = train_test_split(df['text'], df['class'], test_size=0.2)

In [10]:
rev_train.shape, rev_test.shape, class_train.shape, class_test.shape

((1600,), (400,), (1600,), (400,))

In [11]:
# Векторизация Bag of Words
bow = CountVectorizer()
bow.fit_transform(rev_train)
bowed_rev_train = bow.transform(rev_train)  # тренировочные тексты
bowed_rev_test = bow.transform(rev_test)   # тестовые тексты

In [12]:
# Обучение DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(bowed_rev_train, class_train)

# Вывод результатов по Dummy Classifier
print(classification_report(class_test, clf.predict(bowed_rev_test)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       208
          1       0.48      1.00      0.65       192

avg / total       0.23      0.48      0.31       400



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# Обучаем модель логистической регрессии
clf = LogisticRegression(class_weight='balanced')
clf.fit(bowed_rev_train, class_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [14]:
print(classification_report(class_test, clf.predict(bowed_rev_test)))

             precision    recall  f1-score   support

          0       0.85      0.81      0.83       208
          1       0.80      0.84      0.82       192

avg / total       0.83      0.82      0.83       400



In [15]:
# Векторизация TF-IDF
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 0.8,
                             sublinear_tf=True,
                             use_idf=True)
tfidf_rev_train = vectorizer.fit_transform(rev_train)
tfidf_rev_test = vectorizer.transform(rev_test)

In [16]:
# Обучение DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(tfidf_rev_train, class_train)

# Вывод результатов по Dummy Classifier
print(classification_report(class_test, clf.predict(tfidf_rev_test)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       208
          1       0.48      1.00      0.65       192

avg / total       0.23      0.48      0.31       400



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Наша модель справилась с определением сентимента лучше, чем DummyClassifier

In [17]:
# Обучаем модель логистической регрессии
clf = LogisticRegression(class_weight='balanced')
clf.fit(tfidf_rev_train, class_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [18]:
# Печатаем результаты классификации
print(classification_report(class_test, clf.predict(tfidf_rev_test)))

             precision    recall  f1-score   support

          0       0.83      0.82      0.83       208
          1       0.81      0.82      0.81       192

avg / total       0.82      0.82      0.82       400



Наша модель справилась с определением сентимента лучше, чем DummyClassifier

Линейная регрессия

y=β0+β1x

Логистическая регрессия

log(p1−p)=β0+β1x

In [20]:
clf.coef_

array([[-0.06470536,  0.02109143,  0.1232265 , ..., -0.02881488,
        -0.05788145,  0.02541998]])

Положительный коэффициент обозначает, что соответствующий признак увеличивает вероятность класса. Отрицательный коэффициент обозначает, что соответствующий признак уменьшает вероятность класса. Чем больше коэффициент, тем больший вес имеет соответствующий признак. Если коэффициент равен нулю, это означает, что признак не не имеет никакого веса

In [21]:
clf.intercept_

array([-0.05406328])

In [22]:
clf.classes_

array(['0', '1'], dtype=object)

In [33]:
logodds = clf.intercept_ + clf.coef_[0] * 2  # compute predicted log-odds for al=2 using the equation
odds = np.exp(logodds)  # convert log-odds to odds
prob = odds/(1 + odds)  # convert odds to probability

In [46]:
print(tfidf_rev_test[0])

  (0, 11645)	0.0312792732271
  (0, 11633)	0.0335047240612
  (0, 11625)	0.0303885861421
  (0, 11604)	0.0388560941798
  (0, 11580)	0.0771982994605
  (0, 11578)	0.0455144999817
  (0, 11554)	0.0468446658362
  (0, 11434)	0.0932062119881
  (0, 11423)	0.0240497049677
  (0, 11408)	0.0395377216461
  (0, 11346)	0.0232565206957
  (0, 11333)	0.0341694130714
  (0, 11323)	0.0301167887868
  (0, 11303)	0.0885826223185
  (0, 11265)	0.0561080933975
  (0, 11204)	0.0771982994605
  (0, 11200)	0.0448171346934
  (0, 11158)	0.0597025565356
  (0, 11067)	0.0381816653291
  (0, 11044)	0.0180901356533
  (0, 10955)	0.073453456769
  (0, 10893)	0.0699925655796
  (0, 10858)	0.0512896298485
  (0, 10851)	0.0380246467166
  (0, 10804)	0.0430501647531
  :	:
  (0, 1158)	0.151521213019
  (0, 1079)	0.0522397417171
  (0, 1077)	0.0294174072004
  (0, 1066)	0.0285353365673
  (0, 1015)	0.0407330077424
  (0, 977)	0.035353910126
  (0, 974)	0.024398825785
  (0, 941)	0.0680900751713
  (0, 932)	0.0642398673143
  (0, 866)	0.062655229168

In [48]:
# compute predicted probability for al=2 using the predict_proba method
clf.predict_proba(tfidf_rev_test[0])[:, 1]

array([ 0.47900404])

In [45]:
clf.predict(tfidf_rev_test[0])

array(['0'], dtype=object)